## Working directly with the database
The algokitchen database is called **cookbook.db**, to use it we need to load the *sqlitemagic* etension into this notebook, using the statement bellow. The **cookbook.db** is pretty large, around 1Gb in size, so if you want to run small experiments and mess around with a smaller dataset use **cookbook-small.db**.

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of sqlitemagic failed: Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ImportError: No module named sqlitemagic
]


In [11]:
%reload_ext sqlitemagic

## Querying the cookbook
The cookbook has three major tables: **foodstuffs** which contains the names of food items like the ones you would go shopping for. **ingredients** which contains specific quantities and preparations to be included in a recipe. **preparations** which contains the steps necessary to prepare a dish and **recipes** which contains a listing of all known recipes.

To query the cookbook we will be talking directly to the database using a language called [SQL](http://www.sqlcourse.com/intro.html).

### Foodstuffs
Foodstuffs all have a name and *normalized* name. Normalization is often just the name of the ingredient in lowercase letters and with singular words. Get yourself familiar with the *SELECT* syntax, that's the way you can fetch data from the database. The *LIKE* operator matches rows loosely, the *%* character is used to indicate *any text*.

In [12]:
%%sqlite data/cookbook.db
SELECT id, name, normalized FROM foodstuffs WHERE name LIKE "%cheese%";

id,name,normalized
103,"u""goats' cheese""","u""goats' cheese"""
170,u'cream cheese',u'cream cheese'
315,u'cheese',u'cheese'
693,u'halloumi cheese',u'halloumi cheese'
723,u'curd cheese',u'curd cheese'
724,u'smoked cheese',u'smoked cheese'
992,"u""goats' cheeses""","u""goats' cheese"""
993,u'cottage cheese',u'cottage cheese'
996,u'fontina cheese',u'fontina cheese'
1089,u'cheeses',u'cheese'


Try and change the query above for a more specific one matching 'gin'.

In [13]:
%%sqlite data/cookbook.db
SELECT id, name, normalized FROM foodstuffs WHERE name LIKE "gin";

id,name,normalized
290,u'gin',u'gin'
1327,u'Gin',u'gin'


This shows that our database is not so clean after all. We found two matches, but they are in fact the same **foodstuff*.

Also notice that "gin" matches both "Gin" and "gin", so the query is case insensitive.

In [14]:
%%sqlite data/cookbook.db
SELECT id, name FROM recipes LIMIT 10, 10;

id,name
11,u'Barbecue baby back ribs with celeriac slaw'
12,"u""Terrine of Capricorn goats' cheese, apples and celery and a salad of toasted walnuts and raisins"""
13,"u'Roast Mortimer Forest venison loin with bitter chocolate, blackberries and beetroot'"
14,u'Ackee and saltfish'
15,u'Saltfish and ackee with fried dumplings'
16,u'Poached smoked haddock with ackee and wilted callaloo'
17,u'Herb-crusted red snapper with pan-fried ackee and tomato and okra chutney'
18,"u'Steak, peppers and tomatoes with ackee and mushrooms'"
19,u'Ackee and saltfish with steamed callaloo and Merlene\u2019s fried dumplings '
20,u'Roasted pumpkin soup'


In [15]:
%%sqlite data/cookbook.db
SELECT r.id, r.name 
FROM recipes r 
JOIN recipes_ingredients r2i 
ON r2i.recipe_id = r.id 
WHERE r2i.ingredient_id 
IN (
    SELECT i.id 
    FROM ingredients i, 
    foodstuffs fs 
    JOIN ingredients_foodstuffs i2f 
    ON (i2f.ingredient_id = i.id AND i2f.foodstuff_id = fs.id) 
    WHERE fs.name LIKE "apricot"
);

id,name
338,"u'Mincemeat, almond and pear tart'"
546,u'Christmas cake'
981,u'Ultimate rum punch'
990,u'Zombie'
1495,u'The Mayflower'
1515,u'Vermont cocktail'
1570,u'Fred Said'
1794,u'Roasted apricots and basil with apricot and pistachio biscotti and apricot ice cream'
1795,u'Apricot sour'
1796,u'Apricot mousse'
